In [1]:
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path
from typing import List

import utm
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely.geometry as sg
import shapely.ops as so

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
path = DATA_DIR/'cairns_gtfs.zip'
cairns = (
    gk.read_feed(path, dist_units='km')
    .append_dist_to_stop_times()
)
feed = cairns.append_dist_to_stop_times()


In [10]:
dates = feed.get_first_week()[6:]
activity = feed.compute_trip_activity(dates)
display(activity)

trip_ids = activity.loc[
    lambda x: x.filter(dates).sum(axis=1) > 0,
    "trip_id",
]
trip_ids

,trip_id,20140601
0,CNS2014-CNS_MUL-Weekday-00-4165878,0
1,CNS2014-CNS_MUL-Weekday-00-4165879,0
2,CNS2014-CNS_MUL-Weekday-00-4165880,0
3,CNS2014-CNS_MUL-Weekday-00-4165881,0
4,CNS2014-CNS_MUL-Weekday-00-4165882,0
...,...,...
1334,CNS2014-CNS_MUL-Sunday-00-4180878,1
1335,CNS2014-CNS_MUL-Sunday-00-4180879,1
1336,CNS2014-CNS_MUL-Sunday-00-4180880,1
1337,CNS2014-CNS_MUL-Sunday-00-4180881,1


1073    CNS2014-CNS_MUL-Sunday-00-4165971
1074    CNS2014-CNS_MUL-Sunday-00-4165972
1075    CNS2014-CNS_MUL-Sunday-00-4165973
1076    CNS2014-CNS_MUL-Sunday-00-4165974
1077    CNS2014-CNS_MUL-Sunday-00-4165975
                      ...                
1334    CNS2014-CNS_MUL-Sunday-00-4180878
1335    CNS2014-CNS_MUL-Sunday-00-4180879
1336    CNS2014-CNS_MUL-Sunday-00-4180880
1337    CNS2014-CNS_MUL-Sunday-00-4180881
1338    CNS2014-CNS_MUL-Sunday-00-4180994
Name: trip_id, Length: 266, dtype: object

In [29]:
feed.trips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,110-423,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00-4165878,The Pier Cairns Terminus,0,NaN,1100023
1,110-423,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00-4165879,The Pier Cairns Terminus,0,NaN,1100023
2,110-423,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00-4165880,The Pier Cairns Terminus,0,NaN,1100023
3,110-423,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00-4165881,The Pier Cairns Terminus,0,NaN,1100023
4,110-423,CNS2014-CNS_MUL-Weekday-00,CNS2014-CNS_MUL-Weekday-00-4165882,The Pier Cairns Terminus,0,NaN,1100023
...,...,...,...,...,...,...,...
1334,150E-423,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00-4180878,The Pier Cairns Terminus,0,NaN,150E0007
1335,150E-423,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00-4180879,The Pier Cairns Terminus,0,NaN,150E0007
1336,150E-423,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00-4180880,The Pier Cairns Terminus,0,NaN,150E0007
1337,150E-423,CNS2014-CNS_MUL-Sunday-00,CNS2014-CNS_MUL-Sunday-00-4180881,The Pier Cairns Terminus,0,NaN,150E0007


In [35]:
dates = feed.get_first_week()[:1] + ["20010101"]

path = DATA_DIR / "cairns_screen_lines.geojson"
screen_lines = gpd.read_file(path)
f = feed.compute_screen_line_counts(screen_lines, dates)
f

,date,trip_id,route_id,route_short_name,shape_id,screen_line_id,crossing_direction,crossing_distance,crossing_time
0,20140526,CNS2014-CNS_MUL-Weekday-00-4165878,110-423,110,1100023,sl1,1,30.299004,06:40:55
1,20140526,CNS2014-CNS_MUL-Weekday-00-4165878,110-423,110,1100023,sl0,1,31.279921,06:44:29
2,20140526,CNS2014-CNS_MUL-Weekday-00-4165879,110-423,110,1100023,sl1,1,30.299004,07:10:55
3,20140526,CNS2014-CNS_MUL-Weekday-00-4165879,110-423,110,1100023,sl0,1,31.279921,07:14:29
4,20140526,CNS2014-CNS_MUL-Weekday-00-4165880,110-423,110,1100023,sl1,1,30.299004,07:40:55
...,...,...,...,...,...,...,...,...,...
569,20140526,CNS2014-CNS_MUL-Weekday-00-4172816,123-423,123,1230067,sl0,-1,1.496863,14:14:09
570,20140526,CNS2014-CNS_MUL-Weekday-00-4172817,123-423,123,1230067,sl0,-1,1.496863,15:14:09
571,20140526,CNS2014-CNS_MUL-Weekday-00-4172818,123-423,123,1230067,sl0,-1,1.496863,16:14:09
572,20140526,CNS2014-CNS_MUL-Weekday-00-4172819,123-423,123,1230067,sl0,-1,1.496863,17:14:09


In [17]:
path = DATA_DIR/'cairns_gtfs.zip'
gk.list_feed(path)



,file_name,file_size
0,calendar_dates.txt,387
1,routes.txt,1478
2,trips.txt,143081
3,stops.txt,26183
4,agency.txt,199
5,stop_times.txt,2561019
6,shapes.txt,864694
7,calendar.txt,337


In [10]:
feed = (
    gk.read_feed(path, dist_units='km')
    .append_dist_to_stop_times()
)
feed.describe()

,indicator,value
0,agencies,[Department of Transport and Main Roads - Tran...
1,timezone,Australia/Brisbane
2,start_date,20140526
3,end_date,20141228
4,num_routes,22
5,num_trips,1339
6,num_stops,416
7,num_shapes,54
8,sample_date,20140529
9,num_routes_active_on_sample_date,20


In [12]:
shapes = feed.geometrize_shapes(use_utm=True)
shapes.crs
screen_lines = screen_lines.to_crs(shapes.crs)
screen_lines

,geometry
0,"LINESTRING (369339.702 -1870822.603, 369389.75..."
1,"LINESTRING (368695.647 -1870715.321, 368744.51..."


In [5]:
url = "http://transitfeeds.com/p/10-15-transit/936/latest/download"
gk.read_feed(url, dist_units="km").describe()

,indicator,value
0,agencies,[10-15 Transit]
1,timezone,America/Chicago
2,start_date,20171201
3,end_date,20210101
4,num_routes,1
5,num_trips,8
6,num_stops,66
7,num_shapes,1
8,sample_date,20171207
9,num_routes_active_on_sample_date,0


In [ ]:
(
    g1.crossing_time.equals(g2.crossing_time),
    g1.crossing_direction.equals(g2.orientation)
)

In [28]:
feed.map_routes(feed.routes.route_id.iloc[:4], include_stops=False)
